[From Anton Poluektov's "Performing amplitude fits with `TensorFlow`" talk](https://indico.cern.ch/event/613842/contributions/2585806/attachments/1464213/2262841/tfa.pdf#page=6)

In [1]:
import tensorflow as tf
import numpy as np

import itertools # for fast looping
import time # for timing loop
from iminuit import Minuit

In [2]:
x = tf.placeholder(tf.float32, shape = (None))
w = tf.Variable(1.)
p = tf.Variable(0.)
a = tf.Variable(1.)

# Build the computational graph (a function is a graph!)
f = a * tf.sin(w*x + p)

In [3]:
def eval_func(f):
    
    with tf.Session() as sess:
        # Create TF session and initialise variables
        init = tf.global_variables_initializer()
        sess.run(init)
        
        # Run calculation of y by feeding data to tensor x
        return sess.run(f, feed_dict = { x : [1., 2., 3., 4.] })

In [4]:
y_data = eval_func(f)
print(y_data)

[ 0.84147096  0.90929741  0.14112    -0.7568025 ]


Other Examples

In [5]:
mu = 0.
sigma = 3.

# pdf of Gaussian of variable x with mean mu and standard deviation sigma
dist = tf.contrib.distributions.Normal(loc=mu, scale=sigma)

In [6]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Cumulative distribution funtion of pdf evalauted at x=1
    sess.run(dist.cdf(1.))
    
    # Evaluate the pdf at x=0
    sess.run(dist.prob(0.))
    
    sample_data = sess.run(dist.sample(10))
    
    y_data = sess.run(dist.prob(sample_data))

In [7]:
print(y_data)

[ 0.04039098  0.13116929  0.13069689  0.09667946  0.087863    0.0377687
  0.06257927  0.09173018  0.0504996   0.08712515]


In [8]:
def sample_model(model, n_samples):
    x = model.sample(n_samples)
    y = model.prob(x)
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        return sess.run(x), sess.run(y)

In [9]:
test_x, test_y = sample_model(dist, 100)

In [10]:
print(test_x)
print(test_y)

[-1.69987428 -2.01465535  2.51488042 -2.71445441 -0.54834217 -3.38368177
  0.07994295  5.52828121  3.3390305   1.48724782  0.66771591  1.29307485
 -3.40427446 -0.59513313  5.71594858  6.94065714  1.98987257 -1.94839644
 -1.01324224 -0.52712667  0.27350378  0.35006088 -0.65790403 -0.64984143
  1.86532378  1.31571257  0.96918392 -2.01526618  0.19849496  0.53363222
  2.3962605  -5.2568965  -0.13337657  2.12090111  1.09981918  0.26945564
 -5.22269821  1.6723392  -3.11538935  0.70398021 -2.98417568  0.21909946
  2.06357431 -4.49498844 -5.96870804 -0.76383638  2.09158611 -3.58911419
 -0.47039324  0.40047371 -3.66735458  1.19181681  4.61252022 -1.4083091
  1.13313293  3.40650272 -1.26915312 -5.50266361 -1.69950247 -2.55948973
 -1.26903021  4.35879803  0.02533636 -2.7045269   0.39412659 -5.26360846
  0.23553206  1.44785905  1.72671485  1.81495595 -0.3719807  -1.52246642
 -2.12116742 -1.29306281 -0.29624647 -1.74591672  1.6651454   2.30977821
 -3.93077803  2.96089721 -1.99955821  0.41935667 -0.

c.f. https://gist.github.com/ibab/45c3d886c182a1ea26d5

In [11]:
def normal_log(X, mu, sigma, TYPE=np.float32):
    return -tf.log(tf.constant(np.sqrt(2 * np.pi), dtype=TYPE) * sigma) - \
        tf.pow(X - mu, 2) / (tf.constant(2, dtype=TYPE) * tf.pow(sigma, 2))

In [12]:
def nll(X, mu, sigma, TYPE=np.float32):
    return -tf.reduce_sum(normal_log(X, mu, sigma, TYPE))

In [13]:
# MLE attempt
TYPE = np.float64

n_events = 1000000
n_trials = 10

sess = tf.Session()

def func(mu_, sigma_):
    return sess.run(nll_, feed_dict={mu: mu_, sigma: sigma_})

start_time = time.time()
for _ in itertools.repeat(None, n_trials):
    data = np.random.normal(0.5, 1.5, n_events).astype(TYPE)
    
    # Define data as a variable so that it will be cached
    X = tf.Variable(data, name='data')
    
    mu = tf.Variable(TYPE(1), name='mu')
    sigma = tf.Variable(TYPE(2), name='sigma')
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    nll_ = nll(X, mu, sigma, TYPE)
    
    # To guard against excessive output
    if n_trials > 1:
        print_level = 0
    else:
        print_level = 1
    
    minuit = Minuit(func, mu_=10, sigma_=10, error_mu_=0.5, error_sigma_=0.5,
           limit_mu_=(-1, 100), limit_sigma_=(0, 100), errordef=1, print_level=print_level)
    minuit.migrad()
    minuit.minos()
    
end_time = time.time()
time_duration = end_time - start_time
mean_fit_time = time_duration/n_trials

mu_ = minuit.values['mu_']
sigma_ = minuit.values['sigma_']

sess.close()

print("mu = {}, sigma = {}".format(mu_, sigma_))

print("\nfit {} times in {} seconds".format(n_trials, time_duration))
print("The average fit time is {} seconds".format(mean_fit_time))

mu = 0.49853884540774374, sigma = 1.500036754143963

fit 10 times in 10.461521625518799 seconds
The average fit time is 1.04615216255188 seconds
